In [ ]:
pip install python-dotenv

In [ ]:
# import os
# from dotenv import load_dotenv, find_dotenv
# _= load_dotenv(find_dotenv())

# ap =os.getenv("MY_VAR")


In [ ]:
# !pip install langchain
# !pip install openai

In [ ]:
!pip install -U langchain-openai
!pip install langchain


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.0 MB/s eta 0:00:00


In [ ]:
# prompt: 어떻게 api_key를 넣는가?
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv, find_dotenv

# Load the environment variables from the .env file
load_dotenv(find_dotenv())

# Get the OpenAI API key from the environment variables
api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
api_key = 'YOUR API KEY'

In [ ]:
chat=ChatOpenAI(temperature=0,
                openai_api_key=api_key,
                )

Custom Tools

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

template = """Question: {question}
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

llm_chain = LLMChain(
    prompt = prompt,
    llm = chat
)

print(llm_chain.invoke("What are Tesla's revenue in 2022?"))

{'question': "What are Tesla's revenue in 2022?", 'text': 'As of now, Tesla has not released its revenue figures for 2022. The company typically releases its financial results on a quarterly basis, so we will have to wait for their official announcements to know their revenue for 2022.'}


In [ ]:
!pip install langchain-community # Install the langchain_community package

In [ ]:
!pip install duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 51.8 MB/s eta 0:00:00
  Attempting uninstall: lxml
    Found existing installation: lxml 4.9.4
    Uninstalling lxml-4.9.4:
      Successfully uninstalled lxml-4.9.4


In [ ]:
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()
search.invoke("Tesla stock price?")

"The History of Tesla's Stock Price by Markets Insider Tesla launched its IPO on June 29, 2010. Trading on the NASDAQ, Tesla offered 13.3 million shares at a price of $17 per share. See the latest Tesla Inc stock price (NASDAQ:TSLA), related news, valuation, dividends and more to help you make your investing decisions. View Tesla, Inc. TSLA stock quote prices, financial information, real-time forecasts, and company news from CNN. Get a real-time Tesla, Inc. (TSLA) stock price quote with breaking news, financials, statistics, charts and more. 35 Wall Street research analysts have issued 1 year target prices for Tesla's shares. Their TSLA share price targets range from $85.00 to $380.00. On average, they anticipate the company's stock price to reach $219.89 in the next twelve months. This suggests a possible upside of 10.1% from the stock's current price."

In [ ]:
from langchain.tools import Tool

duckduckgo_tool = Tool(
    name = 'DuckDuckGo Search',
    func = search.run,
    description = "Useful for when you need to do a search on the internet to find information that another tool can't find. be specific with your input."
)

In [ ]:
from langchain.agents import AgentType
from langchain.agents import load_tools, initialize_agent

tools = [duckduckgo_tool]

agent = initialize_agent(
    tools,
    chat,
    agent = AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors = True,
    verbose = True
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [ ]:
agent.run("What are Tesla's revenue in 2022?")



> Entering new AgentExecutor chain...
Thought: I should search for Tesla's revenue in 2022 to find the most up-to-date information.
Action:
```
{
  "action": "DuckDuckGo Search",
  "action_input": "Tesla revenue 2022"
}
```

Observation: Tesla's revenue from fiscal year 2008 to fiscal year 2022, based on Statista's data. The electric vehicle maker's revenue growth was driven by the Model 3 and Model Y, and its spending on production and sales in China. Another major source of revenue for Tesla is the sale of regulatory credits to other automotive manufacturers, which brought in $1.8 billion in revenues in 2022. Tesla earns tradable credits under ... Fiscal year is January - December. Source: Financials are provided by Nasdaq Data Link and sourced from the audited annual ( 10-K) and quarterly ( 10-Q) reports submitted to the Securities and Exchange Commission (SEC). Detailed annual and quarterly income statement for Tesla, Inc. (TSLA). See many years of revenue, expenses and profits o

"Tesla's revenue in 2022 was $81.46 billion."

ChatGPT as a Financial Information Extractor

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

extraction_template = """Your task:\n\
Find the value of revenue in the given content.\n\
If you can't find the value, please output "None".\n\

Example 1:\n\
The amount of Apple's annual revenue in 2021 was $365.817B.
Result: 365.817

Given content: {text}
Result:
"""
extraction_prompt = ChatPromptTemplate.from_template(extraction_template)

extraction_chain = LLMChain(
    prompt = extraction_prompt,
    llm = chat
)

In [ ]:
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(chains = [agent, extraction_chain],
                                      verbose = True)

overall_chain.run("What are Tesla's revenue in 2022?")



> Entering new SimpleSequentialChain chain...


> Entering new AgentExecutor chain...
Thought: I should search for Tesla's revenue in 2022 to find the most up-to-date information.
Action:
```
{
  "action": "DuckDuckGo Search",
  "action_input": "Tesla revenue 2022"
}
```

Observation: Tesla's revenue from fiscal year 2008 to fiscal year 2022, based on Statista's data. The electric vehicle maker's revenue growth was driven by the Model 3 and Model Y, and its spending on production and sales in China. Another major source of revenue for Tesla is the sale of regulatory credits to other automotive manufacturers, which brought in $1.8 billion in revenues in 2022. Tesla earns tradable credits under ... Tesla ended Q4 2023 with a net income of $7.9 billion and the full year with $15 billion in profits. While Elon Musk's company more than doubled its earnings between 2021 and 2022, this past year ... 2023 Investor Day 2023 Investor Day Presentation. Cybertruck Delivery Event. 2022. Annual Re

'81.46'

In [ ]:
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(chains = [agent, extraction_chain],
                                      verbose = True)

overall_chain.run("테슬러의 2022년 매출은 얼마인가?")



> Entering new SimpleSequentialChain chain...


> Entering new AgentExecutor chain...
Thought: I need to search for Tesla's revenue for the year 2022.
Action:
```
{
  "action": "DuckDuckGo Search",
  "action_input": "Tesla 2022 revenue"
}
```
Observation: Tesla's annual revenue has grown steadily since 2008, peaking in 2022 after breaking records in 2021. In 2018, Tesla's revenue stood at over 21 billion USD. Another major source of revenue for Tesla is the sale of regulatory credits to other automotive manufacturers, which brought in $1.8 billion in revenues in 2022. Tesla earns tradable credits under ... Fiscal year is January - December. Source: Financials are provided by Nasdaq Data Link and sourced from the audited annual ( 10-K) and quarterly ( 10-Q) reports submitted to the Securities and Exchange Commission (SEC). Detailed annual and quarterly income statement for Tesla, Inc. (TSLA). See many years of revenue, expenses and profits or losses. In 2022, Tesla generated $81.46 bi

'81.46'

# 연습문제

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

template = """Your task:\n\
Find the value of Scope 1 emissions in the given content.\n\
If you can't find the value, please output "None".\n\

Example 1:\n\
TotalEnergies' latest Scope 1 emissions were 32 million metric tons\
 of carbon dioxide in 2021.
Result: 32.0

Given content: {text}
Result:
"""
prompt = ChatPromptTemplate.from_template(template)

llm_chain = LLMChain(
    prompt = prompt,
    llm = chat
)

In [ ]:
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(
    chains = [agent, llm_chain],
    verbose = True
)

In [ ]:
from tqdm.notebook import tqdm

list_companies = ['AT&T',
 'Apple Inc.',
 'Bank of America',
 'Boeing',
 'CVS Health',
 'Chevron Corporation',
 'Cisco',
 'Citigroup',
 'Disney',
 'Dominion Energy',
 'ExxonMobil',
 'Ford Motor Company',
 'General Electric',
 'Home Depot (The)',
 'IBM',
 'Intel',
 'JPMorgan Chase',
 'Johnson & Johnson',
 "Kellogg's",
 'McKesson',
 'Merck & Co.',
 'Microsoft',
 'Oracle Corporation',
 'Pfizer',
 'Procter & Gamble',
 'United Parcel Service',
 'UnitedHealth Group',
 'Verizon',
 'Walmart',
 'Wells Fargo']

list_results = []

for i in tqdm(range(len(list_companies))):
  try:
    response = overall_chain.run(f"What is the amount of {list_companies[i]} Scope 1 emissions?")
  except:
    response = "None"
  list_results.append(response)

  0%|          | 0/30 [00:00<?, ?it/s]



> Entering new SimpleSequentialChain chain...


> Entering new AgentExecutor chain...
Thought: I need to search for the specific information regarding AT&T's Scope 1 emissions.
Action:
```
{
  "action": "DuckDuckGo Search",
  "action_input": "AT&T Scope 1 emissions amount"
}
```
Observation: Achieve carbon neutrality (net zero Scope 1 and 2 emissions 1) by 2035. Progress: 2022 Scope 1 and 2 emissions were approximately 4.8 million MT of CO 2 e. 2 This represents a reduction of more than 4 million MT from our 2015 base year (approximately 8.8 million MT of CO 2 e)—more than 45% 1 attainment toward our net zero target. Science ... Standard Disclosure Disclosure Title Location; GRI 3-3: Management of material topics: See our Climate Change & Greenhouse Gas Emissions issue brief. GRI 305-1: Direct (Scope 1) GHG emissions: Gross direct (Scope 1) GHG emissions in metric tons of CO 2 equivalent: 917,036 MT CO 2 e.. Note that we have reported minor adjustments to historical Scope 1 emissions

In [ ]:
list_results

Sustainability Report as a Knowledge Base

In [ ]:
!pip install unstructured
!pip install pypdf
!pip install pdf2image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 15.3 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=3be44a3223a6028e8f41a342533299f16301b6346747bc510ae31e5ab5b9016b
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 6.3 MB/s eta 0:00:00


In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.document_loaders import OnlinePDFLoader

url = "https://www.tesla.com/ns_videos/2021-tesla-impact-report.pdf"
loader = OnlinePDFLoader(url)
data = loader.load()

KeyboardInterrupt: 

In [ ]:
!pip install docarray
!pip install tiktoken

In [ ]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [ ]:
index.query("What is the Scope 1 Emissions?")